# Convert Tubes To Images

This notebook contains a few examples of how to call wrapped methods in itk and ITKTubeTK.

ITK, ITKTubeTK, and ITKWidgets must be installed on your system for this notebook to work.

In [2]:
import os
import sys
import numpy

In [3]:
import itk
from itk import TubeTK as ttk
from itkwidgets import view

Load the tubes and a reference image which provides the size, spacing, origin, and orientation for the desired output image.

In [4]:
PixelType = itk.F
Dimension = 3
ImageType = itk.Image[PixelType, Dimension]
    
# Read tre file
TubeFileReaderType = itk.SpatialObjectReader[Dimension]
    
tubeFileReader = TubeFileReaderType.New()
tubeFileReader.SetFileName("Data/MRI-Normals/Normal071-VascularNetwork.tre")
tubeFileReader.Update()

tubes = tubeFileReader.GetGroup()


# Read template image
TemplateImageType = itk.Image[PixelType, Dimension]
TemplateImageReaderType = itk.ImageFileReader[TemplateImageType]
    
templateImageReader = TemplateImageReaderType.New()
templateImageReader.SetFileName("Data/MRI-Normals/Normal071-MRA.mha")
templateImageReader.Update()

templateImage = templateImageReader.GetOutput()

Visualize the template image, just because it looks cool - the data in the image is actually irrelevant.

In [5]:
view(templateImage)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

Create a binary image that represents the spatial extent of the TubeSpatialObjects in the hierarchy of SpatialObjects in the variable "tubes" that was read-in above.   If you only want to visualize centerlines of the tubes, set "UseRadius" to false.

In [6]:
TubesToImageFilterType = ttk.ConvertTubesToImage[TemplateImageType]
tubesToImageFilter = TubesToImageFilterType.New()
tubesToImageFilter.SetUseRadius(True)
tubesToImageFilter.SetTemplateImage(templateImageReader.GetOutput())
tubesToImageFilter.SetInput(tubes)
tubesToImageFilter.Update()

outputImage = tubesToImageFilter.GetOutput()

itkImageF4 not loaded from module TubeTK because of exception:
 module 'itk.TubeTKPython' has no attribute 'itkImageF4'
vectoritkImageF4 not loaded from module TubeTK because of exception:
 module 'itk.TubeTKPython' has no attribute 'vectoritkImageF4'


Visualize the results by blending the template and output images.  Again, the content of the template image
doesn't actually matter, but since these tubes were generated from the content of the template image, blending them illustrates how well the binary tube image corresponds with their source image.

In [7]:
TTKImageMathType = ttk.ImageMath[ImageType,ImageType]

imMath = TTKImageMathType.New(Input = outputImage)
imMath.AddImages(templateImage, 2048, 1)
combinedImage = imMath.GetOutput()
view(combinedImage)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…